<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/Acc_Div_SMMC_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Riesgo de Mercado (Simulación Montecarlo)

Existen trés métodos principales para medir el riesgo de mercado
* Simulación histórica
* Paramétrico
* Simulación de Monte Carlo



   **Pérdidas y Ganancias simulación histórica (sin alisado)**
   
Un enfoque más claro es suponer que tenemos:
1.   Una matriz $X_{(n+1)\times m}$ de $m$ factores de riesgo y $n+1$ observaciones. 
2.   Denotemos el vector de precios actual como $X_{00}:=(x_{0,1},x_{0,2},\ldots,x_{0,m})$. 
3.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
4.   Sea $r$ el número de instrumentos de un portafolio, entonces cada instrumento tiene una función de valuación $f_i\colon A_i \to R$ para todo $x\in X$, $i=1,\ldots, r$, donde $A_i\subset X_i$ con $\#(A_i)\leq \#(X_i)$.
5.   Sea $M_{1\times r}=(m_1,\ldots,m_r)$ el vector de posiciones nominales de cada instrumento, es decir, el número de contratos que se tienen por instrumento $m_i\in R$ $(i=1,\ldots,r)$.

In [10]:
!pip install yfinance
import yfinance as yf
import pandas as pd


In [11]:
start = '2020-01-01'

symbol = ['AAPL','MSFT','SQ','AMZN']
stock_data = yf.download(symbol,start)
# stock_data = stock_data.history(start=start)#, end=end) 
# symbol1='APPL'
# stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


[*********************100%***********************]  4 of 4 completed



##Paso 1.
 (Generación de rendimientos) Construir ${\Delta X}_{n\times m}$ que es la matriz de diferencias basados en el operador $T_j$, es decir
		$${\Delta X}_t=\Big[ T_j(\frac{x_{t,1}}{x_{t+1,1}}), T_j(\frac{x_{t,2}}{x_{t+1,2}}),\ldots, T_j(\frac{x_{t,m}}{x_{t+1,m}})\Big]\quad t=0,1,\ldots, n-1.$$

In [12]:
# yf.download?
print(stock_data.Close.head(5))

import numpy as np
X=stock_data.Close
X0=stock_data.Close.tail(1)
DeltaX=np.log(stock_data.Close/stock_data.Close.shift(1))
DeltaX=DeltaX.dropna()

print(DeltaX.head())
print(X0)


                 AAPL         AMZN        MSFT         SQ
Date                                                     
2020-01-02  75.087502  1898.010010  160.619995  63.830002
2020-01-03  74.357498  1874.969971  158.619995  63.000000
2020-01-06  74.949997  1902.880005  159.029999  62.570000
2020-01-07  74.597504  1906.859985  157.580002  64.589996
2020-01-08  75.797501  1891.969971  160.089996  67.599998
                AAPL      AMZN      MSFT        SQ
Date                                              
2020-01-03 -0.009770 -0.012213 -0.012530 -0.013089
2020-01-06  0.007937  0.014776  0.002581 -0.006849
2020-01-07 -0.004714  0.002089 -0.009160  0.031774
2020-01-08  0.015958 -0.007839  0.015803  0.045548
2020-01-09  0.021018  0.004788  0.012416  0.004870
             AAPL         AMZN        MSFT         SQ
Date                                                 
2022-05-11  146.5  2107.439941  260.549988  71.220001


In [13]:
import pandas as pd
M=np.array([1000,5000,2000,100])
print(M)

#Valoración

Y0=X0.squeeze()*M

Y0



[1000 5000 2000  100]


AAPL    1.465000e+05
AMZN    1.053720e+07
MSFT    5.211000e+05
SQ      7.122000e+03
Name: 2022-05-11 00:00:00, dtype: float64

## Simulación

In [14]:
Ns=10000 #Definimos número de simulaciones
alpha=0.98 #Nivel de Confianza para las medidas de riesgo
ChCP=1  #0 Cholesky, 1 Componentes Principales
normempi=0 #0 normal, 1 empírico
eta=.85 #Varianza explicada  (sólo aplica para Componentes Principales)
DeltaX_port=DeltaX
mT=DeltaX_port.shape[1]
print(mT)

print(np.cov(DeltaX_port.T)*(DeltaX_port.shape[1]/(DeltaX_port.shape[1]-1)))

print(((DeltaX_port.shape[1])/(DeltaX_port.shape[1]-1)))


VarDeltaX=np.where(ChCP==0,np.corrcoef(DeltaX_port.T),np.cov(DeltaX_port.T)*((DeltaX_port.shape[1])/(DeltaX_port.shape[1]-1)))


if(ChCP==0):
  CVarDeltaX=np.linalg.cholesky(VarDeltaX)
  M=np.zeros([Ns, mT])
  print((CVarDeltaX.shape[1]))
  print(VarDeltaX.shape[1])
  if normempi==1:
    for i in range(mT):
      M[:,i]=DeltaX_port.iloc[:,i].sort_values(ascending=True).quantile(1-np.random.uniform(0,1,Ns), interpolation = 'higher')
    Met='Cholesky Empírico'
  else:
    for i in range(mT):
      M[:,i]=np.random.normal(np.mean(DeltaX_port.iloc[:,i]), np.std(DeltaX_port.iloc[:,i]), size=(Ns, 1))
    Met='Cholesky Normal'
  DeltaX_s=M.dot(CVarDeltaX) 

4
[[0.0007233  0.00046131 0.00054877 0.00078667]
 [0.00046131 0.00069582 0.00045659 0.00075259]
 [0.00054877 0.00045659 0.00063789 0.00080399]
 [0.00078667 0.00075259 0.00080399 0.00268872]]
1.3333333333333333


In [ ]:
Ns=10000 #Definimos número de simulaciones
alpha=0.98 #Nivel de Confianza para las medidas de riesgo
ChCP=1  #0 Cholesky, 1 Componentes Principales
normempi=1 #0 normal, 1 empírico
eta=.85 #Varianza explicada  (sólo aplica para Componentes Principales)
DeltaX_port=DeltaX
mT=ncol(DeltaX_port)


VarDeltaX=np.where((ChCP==0),np.corrcoef(DeltaX_port),np.var(DeltaX_port)*((DeltaX_port.shape[1])/(DeltaX_port.shape[1]-1)))

if(ChCP==0):
  CVarDeltaX=as.matrix(np.linalg.cholesky(VarDeltaX,pivot=TRUE))
  # CVarDeltaX[is.nan(CVarDeltaX)] <- 0 #quitamos NaN
  # CVarDeltaX[is.na(CVarDeltaX)] <- 0 #quitamos Na
  print((CVarDeltaX.shape[1]))
  print(VarDeltaX.shape[1])
  M=matrix(0, Ns,mT)
  if normempi==1:
    for i in range(mT):
      M[,i]=DeltaX_port.iloc[,i].sort_values(ascending=True).quantile(1-np.random.uniform(-1,0,1000), interpolation = 'higher')
    Met='Cholesky Empírico'
  } else	{
    for (i in 1:mT)
    {
      M[,i]=rnorm(Ns)*sd(DeltaX_port[,i])*(mT/(mT-1))+mean(DeltaX_port[,i])
    }
    Met='Cholesky Normal'
  }
  DeltaX_s=M%*%CVarDeltaX  #Factores de riesgo simulados
} else { #componentes principales
  G<-eigen(VarDeltaX)      #eigenvectores
  g1<-G$values                  #eigenvalores
  g1p=g1/sum(g1)
  
  g1pac=cumsum(g1p) #suma acumulada de varianza
  
  k=which.max(g1pac>eta) #valor mínimo k
  
  y=as.matrix((DeltaX_port-colMeans(DeltaX_port)))%*%G$vectors[,1:k] #componentes principales
  #/sqrt(diag(var(DeltaX))))
  M=matrix(0, Ns,k)
  if(normempi==0)
  {
    for (i in 1:k)
    {
      M[,i]=rnorm(Ns)*sd(y[,i])+mean(y[,i])
    }
    Met='Componentes Principales Normales'
  } else	{
    for (i in 1:k)
    {
      M[,i]=quantile(y[,i],runif(Ns))
    }
    Met='Componentes Principales Empíricos'
  }
  DeltaX_s=M%*% t(G$vectors[,1:k]) #Factores de riesgo simulados
}
Met
DeltaX_port[1:5,]


Met


Ms1='Se explica el' 
Ms2= 'de varianza'
Ms3= 'de suma de var-cov'

Ms1
print(sum(diag(var(DeltaX_s)))/sum(diag(var(DeltaX_port)))) #Comprobación de matriz de varianza-cov vs original
Ms2

Ms1
print(matrix(1,1,ncol(DeltaX_s))%*%var(DeltaX_s)%*%t(matrix(1,1,ncol(DeltaX_s)))/matrix(1,1,ncol(DeltaX_s))%*%var(DeltaX_port)%*%t(matrix(1,1,ncol(DeltaX_s)))) #Comprobación de matriz de varianza-cov vs original
Ms3

print(mT)
print(k) #número de variables simuladas
#print(g1pac)

##Paso 2. 
Construir ${X^\ast}_{n\times m}$ que es la matriz de factores de riesgo simulada basada en el vector de precios actual $X_{00}$
		$$X_{t}^\ast=\Big[x_{0,1}T^{-1}_j(\Delta x_{t,1}), x_{0,2}T^{-1}_j(\Delta x_{t,2}),\ldots, x_{0,m}T^{-1}_j(\Delta x_{t,m})\Big]\quad (t=1,\ldots,n\quad j=1,2),$$
		donde $T_1(x)=\ln(x)$, $T_2(x)=x-1$.



In [ ]:
X_s=np.exp(DeltaX.squeeze())*X0.squeeze()
X_s.head

<bound method NDFrame.head of                   AAPL         AMZN        MSFT          SQ
Date                                                       
2020-01-02         NaN          NaN         NaN         NaN
2020-01-03  176.041718  3285.645295  309.951879  138.278241
2020-01-06  179.186519  3375.529790  314.671256  139.143767
2020-01-07  176.933944  3332.976577  310.998286  144.622965
2020-01-08  180.629664  3300.048272  318.859267  146.628901
...                ...          ...         ...         ...
2022-03-24  181.801450  3330.935610  318.691197  139.748208
2022-03-25  178.433814  3348.864228  313.426492  135.193502
2022-03-28  178.665371  3411.141967  321.115331  148.473965
2022-03-29  181.171526  3332.406721  318.617880  148.945012
2022-03-30  176.587915  3266.813044  312.317584  133.669383

[566 rows x 4 columns]>

## Paso 3. 
Construcción de la matriz de reevaluación del portafolio basada en los escenarios históricos $Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$Y_t=[m_1f_1(X_{t}^\ast), m_2f_2(X_{t}^\ast),\ldots, m_rf_r(X_{t}^\ast)]\quad (t=1,\ldots,n).$$
    


In [ ]:
Y_s=X_s*M.squeeze()
Y_s.head


<bound method NDFrame.head of                      AAPL          AMZN           MSFT            SQ
Date                                                                
2020-01-02            NaN           NaN            NaN           NaN
2020-01-03  176041.717977  1.642823e+07  619903.758653  13827.824113
2020-01-06  179186.519259  1.687765e+07  629342.511963  13914.376729
2020-01-07  176933.943758  1.666488e+07  621996.572295  14462.296508
2020-01-08  180629.663807  1.650024e+07  637718.534353  14662.890131
...                   ...           ...            ...           ...
2022-03-24  181801.449534  1.665468e+07  637382.393891  13974.820779
2022-03-25  178433.814308  1.674432e+07  626852.983491  13519.350201
2022-03-28  178665.370977  1.705571e+07  642230.661433  14847.396522
2022-03-29  181171.526494  1.666203e+07  637235.759509  14894.501238
2022-03-30  176587.914805  1.633407e+07  624635.168581  13366.938300

[566 rows x 4 columns]>

## Paso 4. 
Construcción de la matriz de pérdidas y ganancias del portafolio basada en los escenarios históricos $\Delta Y_{n\times r}$ de todos los instrumentos financieros, es decir
		$$\Delta Y_t=Y_t-Y_{0}=[m_1(f_1(X_{00})-f_1(X_{t}^\ast)), m_2( f_2(X_{00})-f_2(X_{t}^\ast)),\ldots, m_r(f_r(X_{00})-f_r(X_{t}^\ast))],$$
		para $(t=1,\ldots,n)$. Se puede obtener el vector de pérdidas totales ${\Delta YT}_{n\times m}$ muy fácilmente, esto es $\Delta YT=\sum_{k=1}^r Y_{kt}$ $(t=1,\ldots,n)$, incluso se puede hacer lo mismo por tipo de riesgo.


In [ ]:
PL=Y_s-Y0

cols=PL.shape[1]

PL['Total']=PL.sum(axis=1)
PL.head

<bound method NDFrame.head of                    AAPL           AMZN          MSFT          SQ  \
Date                                                               
2020-01-02          NaN            NaN           NaN         NaN   
2020-01-03 -1728.286296 -201873.622316  -7816.212051 -182.176497   
2020-01-06  1416.514987  247548.851123   1622.541260  -95.623882   
2020-01-07  -836.060514   34782.788937  -5723.398408  452.295898   
2020-01-08  2859.659535 -129858.739478   9998.563650  652.889521   
...                 ...            ...           ...         ...   
2022-03-24  4031.445261   24577.953606   9662.423188  -35.179831   
2022-03-25   663.810036  114221.041465   -866.987212 -490.650409   
2022-03-28   895.366705  425609.735891  14510.690730  837.395912   
2022-03-29  3401.522222   31933.506724   9515.788806  884.500628   
2022-03-30 -1182.089468 -296034.877785  -3084.802122 -643.062310   

                    Total  
Date                       
2020-01-02       0.000000  
2

##Paso 5. 
Obtenerla medida de riesgo basado en un nivel de confianza de la matriz ${\Delta Y}_{\cdot k}$ $(k=1,\ldots,r)$ y $\Delta YT$.

Las pérdidas y ganancias vía simulación histórica sin alisado es donde se ponderan los escenarios históricos con un mismo peso para todos, es decir cada escenario tiene una probabilidad de ocurrencia de $1/n$.

In [ ]:
alpha = 0.98 #@param {type:"slider", min:0.8, max:1, step:0.001}

###Valor en Riesgo (VaR)

Es un cuantil a un $\alpha$ de confianza de la distribución de pérdidas y ganancias, generalmente se expresa en términos monetarios.

#VaR

El valor en riesgo (VaR) es una estadística utilizada para tratar de cuantificar el nivel de riesgo financiero dentro de una empresa o cartera durante un período de tiempo específico. VaR proporciona una estimación de la pérdida máxima de una posición o cartera determinada durante un período de tiempo, y puede calcularla a través de varios niveles de confianza.

Estimar el riesgo de una cartera es importante para el crecimiento del capital a largo plazo y la gestión del riesgo, en particular dentro de empresas o instituciones más grandes. El VaR generalmente se enmarca como algo como esto:

“Tenemos un VaR de cartera de 250.000 USD para el próximo mes al 95% de confianza”
Esto significa que, con un 95% de confianza, podemos decir que la pérdida de la cartera no superará los 250.000 USD en un mes.
En esta publicación, lo guiaré a través de los pasos para calcular esta métrica en una cartera de acciones.

In [ ]:
VaR=PL.quantile(1-alpha, numeric_only=False)

print(VaR)


VaRsi=[]

for i in range(0,PL.shape[1]):
  VC=PL.iloc[:,i].sort_values(ascending=True).quantile(1-alpha, interpolation = 'higher')
  VaRsi.append(VC)

print(VaRsi)

print(sum(Y0))

AAPL      -8193.309543
AMZN    -758814.358453
MSFT     -26925.092186
SQ        -1109.632491
Total   -809806.864573
Name: 0.020000000000000018, dtype: float64
[-8091.457725118351, -730799.392254062, -26566.717137969215, -1052.1554296586746, -777416.0683959855]
17449600.073242188


###Valor en Riesgo Condicional o "Expected Shortfall"

Es el promedio de las pérdidas que son más severas que el Valor en Riesgo ($VaR_\alpha$) a un $\alpha$ de confianza, es decir:
$$E(x|x\le VaR_\alpha)$$


In [ ]:
def VaR_Cond(x , VaR ):
  return np.mean(x[x <= VaR])

In [ ]:

VaR_C=[]

for i in range(0,PL.shape[1]):
  VC=VaR_Cond(PL.iloc[:,i],VaR[i])
  VaR_C.append(VC)


In [ ]:

print(VaR)
print(VaR_C)

pd.concat([VaR,pd.DataFrame(VaR_C).squeeze()], axis=1, ignore_index=True)

AAPL      -8193.309543
AMZN    -758814.358453
MSFT     -26925.092186
SQ        -1109.632491
Total   -809806.864573
Name: 0.020000000000000018, dtype: float64
[-12342.591980741228, -1012696.0297471011, -40780.820161650394, -1627.0165123262495, -1054330.3999173671]


,0,1
AAPL,-8193.309543,NaN
AMZN,-758814.358453,NaN
MSFT,-26925.092186,NaN
SQ,-1109.632491,NaN
Total,-809806.864573,NaN
0,NaN,-1.234259e+04
1,NaN,-1.012696e+06
2,NaN,-4.078082e+04
3,NaN,-1.627017e+03
4,NaN,-1.054330e+06
